<a href="https://colab.research.google.com/github/Hidenori24/Signate_colab/blob/main/SMBC_GreenChallenge_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## importとシードなどの設定

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
# 日本語フォントを簡単に使う
!pip -q install japanize-matplotlib
import japanize_matplotlib

# シードの設定
np.random.seed(42)
tf.random.set_seed(42)

## Driveへの接続、データの取得

In [55]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/ML/Signate_1634/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### train/test/sample_submission の取り込み

In [56]:
train_df  = pd.read_csv(data_path + 'train.csv', parse_dates=['time'], index_col='time')
df = train_df
test_df   = pd.read_csv(data_path + 'test.csv', parse_dates=['time'], index_col='time')
sample_submission_df = pd.read_csv(data_path + 'sample_submit.csv', header=None)  # header 無し

### 概要確認

In [57]:
# 概要を確認
print("\n--- 学習データ (train_df) ---")
display(train_df.head())
print(f"shape: {train_df.shape}")

print("\n--- テストデータ (test_df) ---")
display(test_df.head())
print(f"shape: {test_df.shape}")

print("\n--- サンプル提出 (sample_submission_df) ---")
display(sample_submission_df.head())
print(f"shape: {sample_submission_df.shape}")


--- 学習データ (train_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,price_actual,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,64.02,spain_total
2015-01-02 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,58.46,spain_total
2015-01-02 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.70,spain_total
2015-01-02 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,54.91,spain_total
2015-01-02 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,...,0.0,0.0,0,0,800,clear,sky is clear,01n,53.07,spain_total


shape: (26280, 92)

--- テストデータ (test_df) ---


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run_of_river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,seville_wind_deg,seville_rain_1h,seville_rain_3h,seville_snow_3h,seville_clouds_all,seville_weather_id,seville_weather_main,seville_weather_description,seville_weather_icon,item_ID
time,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+01:00,279.0,0.0,3927.0,895.0,189.0,230.0,1069.0,1893.0,7104.0,53.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 01:00:00+01:00,282.0,0.0,3948.0,878.0,177.0,1269.0,1058.0,1024.0,7101.0,52.0,...,343,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 02:00:00+01:00,283.0,0.0,3791.0,890.0,175.0,2197.0,1052.0,888.0,7100.0,52.0,...,0,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 03:00:00+01:00,280.0,0.0,3671.0,881.0,175.0,2965.0,1032.0,645.0,7101.0,53.0,...,40,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total
2018-01-01 04:00:00+01:00,286.0,0.0,3460.0,861.0,173.0,2705.0,1001.0,661.0,7101.0,53.0,...,30,0.0,0.0,0,0,800,clear,sky is clear,01n,spain_total


shape: (8760, 91)

--- サンプル提出 (sample_submission_df) ---


,0,1
0,2018-01-01 00:00:00+01:00,0
1,2018-01-01 01:00:00+01:00,0
2,2018-01-01 02:00:00+01:00,0
3,2018-01-01 03:00:00+01:00,0
4,2018-01-01 04:00:00+01:00,0


shape: (8760, 2)


## データの前処理

In [58]:
# 基本情報を確認
print(f"データのサイズ: {df.shape}")
print(f"期間: {df.index.min()} から {df.index.max()}")
print(f"⽋損値の合計: {df.isna().sum().sum()}")
# ⽋損値を処理
# 時系列データなので、前⽅補間が適切
#df = df.fillna(method='ffill')
df = df.ffill()
# まだ⽋損がある場合は後⽅補間
#df = df.fillna(method='bfill')
df = df.bfill()
# ⽬的変数の確認
print("\n⽬的変数 'price_actual' の統計:")
print(df['price_actual'].describe())
# 価格の時系列プロット
plt.figure(figsize=(15, 6))
plt.plot(df.index, df['price_actual'])
plt.title('電⼒価格の推移')
plt.ylabel('価格')
plt.grid(True)
plt.tight_layout()
plt.savefig('price_time_series.png')
plt.close()
# 説明変数を準備
# item_IDはスペインの合計データのみ使⽤
df = df[df['item_ID'] == 'spain_total']
# 不要な列を削除
X = df.drop(['price_actual', 'item_ID'], axis=1)
# ⽬的変数
y = df['price_actual']

データのサイズ: (26280, 92)
期間: 2015-01-02 00:00:00+01:00 から 2017-12-31 23:00:00+01:00
⽋損値の合計: 321

⽬的変数 'price_actual' の統計:
count    26280.000000
mean        56.028338
std         14.340356
min          9.330000
25%         47.617500
50%         55.930000
75%         65.192500
max        116.800000
Name: price_actual, dtype: float64


## 特徴量エンジニアリング

In [59]:
def create_additional_features(df):
    """
    時間ベースの特徴量とラグ特徴量を作成
    """
    df_features = df.copy()

    # インデックスがDatetimeIndexであることを確認し、必要なら変換
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # ここから既存の時間ベースの特徴量作成コード
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    # ISO週番号の取得方法が古いバージョンのPandasで異なる場合があります。
    # .weekofyear は非推奨になったため、.isocalendar().week を使用するのが推奨です。
    # ただし、もし古いPandasバージョンでisocalendarが使えない場合は .weekofyear を試してください。
    try:
        df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int) # ISO週番号
    except AttributeError:
        # もし isocalendar がない場合は、古い .weekofyear を試す
        print("警告: .isocalendar() が見つかりません。代わりに .weekofyear を使用します。Pandasのバージョンを確認してください。")
        df_features['weekofyear'] = df_features.index.weekofyear.astype(int)

    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量（例: 1時間前、24時間前、48時間前、1週間前）
    # ... (既存のラグ特徴量作成コード)
    df_features['price_actual_lag_1'] = df_features['price_actual'].shift(1)
    df_features['price_actual_lag_24'] = df_features['price_actual'].shift(24)
    df_features['price_actual_lag_48'] = df_features['price_actual'].shift(48)
    df_features['price_actual_lag_168'] = df_features['price_actual'].shift(168) # 1週間前

    # 必要に応じて他の特徴量のラグも追加できます。
    # 例: df_features['consumption_lag_24'] = df_features['consumption'].shift(24)

    # 移動平均（例: 24時間の移動平均）
    df_features['price_actual_rolling_mean_24'] = df_features['price_actual'].rolling(window=24).mean()

    # 指数平滑化（例: span=24の指数平滑化）
    #df_features['price_actual_ewm_24'] = df_features['price_actual'].ewm(span=24).mean()


    # 新たに生成された特徴量によって発生する可能性のある欠損値を処理
    df_features = df_features.ffill()
    df_features = df_features.bfill()

    return df_features

## 特徴量選択と次元削減

In [60]:
# 特徴量選択（f_regressionを使って上位の特徴を選択）
def select_features(X, y, k=30):
  # 数値型以外のカラムを削除
  X_numeric = X.select_dtypes(include=np.number)
  # もし数値型カラムがkより少ない場合はkを調整
  k = min(k, X_numeric.shape[1])

  selector = SelectKBest(score_func=f_regression, k=k)
  X_selected = selector.fit_transform(X_numeric, y)
  # 選択された特徴のインデックスを取得
  selected_indices = selector.get_support(indices=True)
  selected_features = X_numeric.columns[selected_indices] # X_numericのカラム名を使用
  print(f"選択された特徴量 ({k}個): {selected_features.tolist()}")
  # 選択された特徴をデータフレームとして返す
  return X_numeric.iloc[:, selected_indices] # X_numericから選択

# PCAで次元削減
def apply_pca(X, n_components=15):
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  pca = PCA(n_components=n_components)
  X_pca = pca.fit_transform(X_scaled)
  # 説明された分散の⽐率を確認
  explained_variance = pca.explained_variance_ratio_.sum()
  print(f"PCAにより{n_components}成分で{explained_variance:.2%}の分散を説明")
  return X_pca, pca, scaler

# 特徴量エンジニアリングを適用
df_featured = create_additional_features(df.copy()) # create_additional_features関数を呼び出し

# 特徴量エンジニアリング後のデータから説明変数と目的変数を作成
X_featured = df_featured.drop(['price_actual', 'item_ID'], axis=1) # item_IDも削除が必要
y_featured = df_featured['price_actual']

# 特徴量選択と次元削減を適⽤
X_selected = select_features(X_featured, y_featured, k=30)
X_pca, pca, pca_scaler = apply_pca(X_selected, n_components=15)
# 相関ヒートマップ（選択された特徴量）
plt.figure(figsize=(15, 12))
# X_selected が DataFrame であることを確認
if isinstance(X_selected, pd.DataFrame):
    correlation = X_selected.corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))
    sns.heatmap(correlation, mask=mask, annot=False, cmap='coolwarm', linewidths=0.5)
    plt.title('選択された特徴量の相関ヒートマップ')
    plt.tight_layout()
    plt.savefig('feature_correlation.png')
    plt.close()
else:
    print("X_selected is not a DataFrame, cannot generate correlation heatmap.")

選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_other', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'hour', 'dayofweek', 'month', 'quarter', 'dayofyear', 'weekofyear', 'is_weekend', 'price_actual_lag_1', 'price_actual_lag_24', 'price_actual_lag_48', 'price_actual_lag_168', 'price_actual_rolling_mean_24']
PCAにより15成分で88.59%の分散を説明


## 時系列データの準備

In [61]:
def create_sequences(X, y, seq_length=24):
    """
    時系列データのシーケンスを作成
    seq_length: 何時間前までのデータを使うか（例：24時間）
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# 時系列分割（単純に過去と未来で分けるのではなく、時系列分割を使用）
# 全体の80%をトレーニング、10%を検証、10%をテストに使用
train_size = int(len(X_pca) * 0.8)
val_size = int(len(X_pca) * 0.1)

# スケーリング
y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y_featured.values.reshape(-1, 1))

# トレーニング、検証、テストデータの分割
X_train = X_pca[:train_size]
X_val = X_pca[train_size:train_size+val_size]
X_test = X_pca[train_size+val_size:]

y_train = y_scaled[:train_size]
y_val = y_scaled[train_size:train_size+val_size]
y_test = y_scaled[train_size+val_size:]

# シーケンスデータの作成
seq_length = 24  # 24時間（1日）のデータを使って予測
X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

print(f"トレーニングデータ形状: {X_train_seq.shape}, {y_train_seq.shape}")
print(f"検証データ形状: {X_val_seq.shape}, {y_val_seq.shape}")
print(f"テストデータ形状: {X_test_seq.shape}, {y_test_seq.shape}")

トレーニングデータ形状: (21000, 24, 15), (21000, 1)
検証データ形状: (2604, 24, 15), (2604, 1)
テストデータ形状: (2604, 24, 15), (2604, 1)


## モデル構築と学習

### LSTMモデル

In [62]:
def build_lstm_model(seq_length, n_features):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# LSTMモデルの構築
lstm_model = build_lstm_model(seq_length, X_train_seq.shape[2])
print(lstm_model.summary())

# コールバックの設定
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    verbose=1,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'lstm_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# モデルの学習
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[early_stopping, checkpoint, reduce_lr],
    verbose=1
)

# 学習履歴のプロット部分
# 履歴に'mae'がなくなるため、プロットコードも修正が必要です。
# lossのプロットはそのまま使用できます。
plt.figure(figsize=(7, 5)) # MAEプロットを削除したため、図のサイズを調整

plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('lstm_training_history_loss.png') # ファイル名を変更
plt.close()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 24, 64)         │        20,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,225 (485.25 KB)

 Trainable params: 123,841 (483.75 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0042 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 37/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0041
Epoch 37: val_loss did not improve from 0.00105
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0041 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 38/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0042
Epoch 38: val_loss did not improve from 0.00105
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0042 - val_loss: 0.0012 - learning_rate: 1.0000e-04
Epoch 39/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0041
Epoch 39: val_loss did not improve from 0.00105
329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0041 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 40/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0040
Epoch 40: val_loss did not improve from 0.00105
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0040 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 41/100
3

329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0039 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 43/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0038
Epoch 43: val_loss improved from 0.00105 to 0.00104, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0038 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 44/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0038
Epoch 44: val_loss did not improve from 0.00104
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0038 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 45/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0038
Epoch 45: val_loss did not improve from 0.00104
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0038 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 46/100
324/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0038
Epoch 46: val_loss improved from 0.00104 to 0.00101, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0038 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 47/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037
Epoch 47: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0037 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 48/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0038
Epoch 48: val_loss did not improve from 0.00101
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0038 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 49/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 49: val_loss improved from 0.00101 to 0.00097, saving model to lstm_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0037 - val_loss: 9.7296e-04 - learning_rate: 1.0000e-04
Epoch 50/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037
Epoch 50: val_loss did not improve from 0.00097
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0037 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 51/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037
Epoch 51: val_loss did not improve from 0.00097
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0037 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 52/100
326/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0037
Epoch 52: val_loss did not improve from 0.00097
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0037 - val_loss: 0.0010 - learning_rate: 1.0000e-04
Epoch 53/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037
Epoch 53: val_loss did not improve from 0.00097
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0037 - val_loss: 0.0011 - learning_rate: 1.0000e-04
Epoch 54/100

### GRU モデル（比較用）

In [63]:
def build_gru_model(seq_length, n_features):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(seq_length, n_features)),
        BatchNormalization(),
        Dropout(0.3),

        GRU(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# GRUモデルの構築と学習
gru_model = build_gru_model(seq_length, X_train_seq.shape[2])
print(gru_model.summary())

gru_history = gru_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=64,
    validation_data=(X_val_seq, y_val_seq),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        ModelCheckpoint('gru_model.h5', monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
    ],
    verbose=1
)

# GRU 学習履歴のプロット部分も同様に修正
plt.figure(figsize=(7, 5)) # MAEプロットを削除したため、図のサイズを調整

plt.plot(gru_history.history['loss'], label='Training Loss')
plt.plot(gru_history.history['val_loss'], label='Validation Loss')
plt.title('GRU - Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('gru_training_history_loss.png') # ファイル名を変更
plt.close()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                     │ (None, 24, 64)         │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 24, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 128)            │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 94,977 (371.00 KB)

 Trainable params: 94,593 (369.50 KB)

 Non-trainable params: 384 (1.50 KB)

...

329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0099 - val_loss: 0.0019 - learning_rate: 0.0010
Epoch 5/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0084
Epoch 5: val_loss did not improve from 0.00188
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0084 - val_loss: 0.0022 - learning_rate: 0.0010
Epoch 6/100
323/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0074
Epoch 6: val_loss did not improve from 0.00188
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0074 - val_loss: 0.0024 - learning_rate: 0.0010
Epoch 7/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0075
Epoch 7: val_loss improved from 0.00188 to 0.00178, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0075 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 8/100
328/329 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0065
Epoch 8: val_loss improved from 0.00178 to 0.00169, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0065 - val_loss: 0.0017 - learning_rate: 0.0010
Epoch 9/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0063
Epoch 9: val_loss improved from 0.00169 to 0.00148, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0063 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 10/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0064
Epoch 10: val_loss improved from 0.00148 to 0.00121, saving model to gru_model.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0064 - val_loss: 0.0012 - learning_rate: 0.0010
Epoch 11/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0062
Epoch 11: val_loss did not improve from 0.00121
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0062 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 12/100
325/329 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0063
Epoch 12: val_loss did not improve from 0.00121
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0063 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 13/100
327/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0058
Epoch 13: val_loss did not improve from 0.00121
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0058 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 14/100
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0060
Epoch 14: val_loss did not improve from 0.00121
329/329 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0060 - val_loss: 0.0027 - learning_rate: 0.0010
Epoch 15/100
328/329 ━━━━━━━━━━━━━━

## モデル評価と予測

In [64]:
def evaluate_model(model, X_test, y_test, scaler, model_name):
    # 予測
    y_pred = model.predict(X_test)

    # スケーリングを元に戻す
    y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_orig = scaler.inverse_transform(y_pred).flatten()

    # 評価指標の計算
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
    r2 = r2_score(y_test_orig, y_pred_orig)

    print(f"\n{model_name} モデルの評価:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")

    # 予測と実際の値のプロット
    plt.figure(figsize=(15, 6))
    plt.plot(y_test_orig, label='実際の価格', color='blue')
    plt.plot(y_pred_orig, label='予測価格', color='red', alpha=0.7)
    plt.title(f'{model_name} - 予測 vs 実際の価格')
    plt.xlabel('時間')
    plt.ylabel('価格')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{model_name.lower()}_predictions.png')
    plt.close()

    return y_test_orig, y_pred_orig

# モデルの評価
lstm_y_test, lstm_y_pred = evaluate_model(lstm_model, X_test_seq, y_test_seq, y_scaler, 'LSTM')
gru_y_test, gru_y_pred = evaluate_model(gru_model, X_test_seq, y_test_seq, y_scaler, 'GRU')

# モデル比較
plt.figure(figsize=(15, 6))
plt.plot(lstm_y_test[:100], label='実際の価格', color='blue')
plt.plot(lstm_y_pred[:100], label='LSTM 予測', color='red', alpha=0.7)
plt.plot(gru_y_pred[:100], label='GRU 予測', color='green', alpha=0.7)
plt.title('モデル比較 (最初の100時間)')
plt.xlabel('時間')
plt.ylabel('価格')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('model_comparison.png')
plt.close()

82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

LSTM モデルの評価:
MAE: 4.51
RMSE: 6.34
R²: 0.7027
82/82 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step

GRU モデルの評価:
MAE: 5.03
RMSE: 6.88
R²: 0.6499


## 特徴量重要度の分析

In [65]:
# 特徴量重要度の可視化（SelectKBestの結果）
def plot_feature_importance(X, y, top_n=20):
    # 数値型以外のカラムを削除
    X_numeric = X.select_dtypes(include=np.number)

    selector = SelectKBest(score_func=f_regression, k='all')
    # 数値型カラムのみを使ってfitを行う
    selector.fit(X_numeric, y)

    # 特徴量のスコアを取得
    feature_scores = pd.DataFrame({
        'Feature': X_numeric.columns, # X_numericのカラム名を使用
        'Score': selector.scores_
    })

    # スコア順にソート
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    # 上位n個の特徴量をプロット
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Score', y='Feature', data=feature_scores.head(top_n))
    plt.title(f'上位{top_n}個の重要な特徴量')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

    return feature_scores

# 特徴量重要度のプロット
# XがDataFrameであり、数値型以外のカラムを含んでいる可能性があるため、関数内で処理します
importance = plot_feature_importance(X_featured, y_featured, top_n=20)
print("\n上位20の重要な特徴量:")
print(importance.head(20))


上位20の重要な特徴量:
                                        Feature          Score
84                           price_actual_lag_1  362397.475584
85                          price_actual_lag_24   55455.839343
88                 price_actual_rolling_mean_24   51125.358795
87                         price_actual_lag_168   46096.371945
86                          price_actual_lag_48   29553.563560
3                   generation_fossil_hard_coal    7410.283193
1          generation_fossil_brown_coal/lignite    4315.279936
14                            total_load_actual    4191.590208
2                         generation_fossil_gas    4151.326152
5   generation_hydro_pumped_storage_consumption    2642.416186
6    generation_hydro_run_of_river_and_poundage    1905.413869
79                                      quarter    1801.790531
4                         generation_fossil_oil    1735.021199
75                                         hour    1639.636595
81                                   week

## 時系列交差検証

In [66]:
# 時系列交差検証
def time_series_cv_evaluation(X, y, seq_length=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    mae_scores = []
    rmse_scores = []
    fold = 1

    for train_index, test_index in tscv.split(X):
        print(f"\n評価 fold {fold}/{n_splits}")

        # データの分割
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # スケーリング
        y_scaler_fold = MinMaxScaler()
        y_train_scaled = y_scaler_fold.fit_transform(y_train_fold.reshape(-1, 1))
        y_test_scaled = y_scaler_fold.transform(y_test_fold.reshape(-1, 1))

        # シーケンスデータの作成
        X_train_seq_fold, y_train_seq_fold = create_sequences(X_train_fold, y_train_scaled, seq_length)
        X_test_seq_fold, y_test_seq_fold = create_sequences(X_test_fold, y_test_scaled, seq_length)

        if len(X_train_seq_fold) == 0 or len(X_test_seq_fold) == 0:
            print("シーケンスデータが空です")
            continue

        # モデル構築と学習
        model = build_lstm_model(seq_length, X_train_seq_fold.shape[2])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

        model.fit(
            X_train_seq_fold, y_train_seq_fold,
            epochs=50,
            batch_size=64,
            validation_split=0.2,
            callbacks=[early_stopping],
            verbose=0
        )

        # 評価
        y_pred_fold = model.predict(X_test_seq_fold)

        # スケーリングを元に戻す
        y_test_orig_fold = y_scaler_fold.inverse_transform(y_test_seq_fold.reshape(-1, 1)).flatten()
        y_pred_orig_fold = y_scaler_fold.inverse_transform(y_pred_fold).flatten()

        # 評価指標の計算
        mae = mean_absolute_error(y_test_orig_fold, y_pred_orig_fold)
        rmse = np.sqrt(mean_squared_error(y_test_orig_fold, y_pred_orig_fold))

        mae_scores.append(mae)
        rmse_scores.append(rmse)

        print(f"Fold {fold} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
        fold += 1

    # 結果をまとめる
    print("\n時系列交差検証の結果:")
    print(f"平均 MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")
    print(f"平均 RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")

    return mae_scores, rmse_scores

# 時系列交差検証の実行
mae_cv, rmse_cv = time_series_cv_evaluation(X_pca, y_scaled, seq_length=24, n_splits=3)


評価 fold 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Fold 1 - MAE: 0.09, RMSE: 0.11

評価 fold 2/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Fold 2 - MAE: 0.07, RMSE: 0.09

評価 fold 3/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Fold 3 - MAE: 0.04, RMSE: 0.05

時系列交差検証の結果:
平均 MAE: 0.07 (±0.02)
平均 RMSE: 0.09 (±0.03)


## モデルとパイプラインの保存

In [67]:
import pickle

# モデル保存のためのパイプラインを作成
pipeline = {
    'pca': pca,
    'pca_scaler': pca_scaler,
    'y_scaler': y_scaler,
    'selected_features': X_selected.columns.tolist(),
    'seq_length': seq_length
}

# パイプラインを保存
pipeline_save_path = data_path + 'price_prediction_pipeline.pkl'
with open(pipeline_save_path, 'wb') as f:
    pickle.dump(pipeline, f)

# 予測パイプラインの使用例
def predict_price(model, new_data, pipeline):
    """
    新しいデータで価格を予測する

    Parameters:
    - model: 訓練済みモデル
    - new_data: 予測したいデータ（必要な特徴量を含むDataFrame）
    - pipeline: 前処理パイプライン

    Returns:
    - predicted_price: 予測された価格
    """
    # パイプラインから必要な要素を取得
    pca = pipeline['pca']
    pca_scaler = pipeline['pca_scaler']
    y_scaler = pipeline['y_scaler']
    selected_features = pipeline['selected_features']
    seq_length = pipeline['seq_length']

    # 必要な特徴量のみを選択
    selected_data = new_data[selected_features]

    # スケーリングとPCA変換
    scaled_data = pca_scaler.transform(selected_data)
    pca_data = pca.transform(scaled_data)

    # シーケンスデータの作成（最新のseq_length分のデータを使用）
    if len(pca_data) >= seq_length:
        sequence = pca_data[-seq_length:].reshape(1, seq_length, -1)

        # 予測
        scaled_prediction = model.predict(sequence)

        # スケーリングを元に戻す
        prediction = y_scaler.inverse_transform(scaled_prediction)[0, 0]

        return prediction
    else:
        raise ValueError(f"入力データが短すぎます。少なくとも{seq_length}点必要です。")

print("\nモデルと前処理パイプラインを保存しました。")
print("新しいデータに対する予測は 'predict_price' 関数を使用して行えます。")


モデルと前処理パイプラインを保存しました。
新しいデータに対する予測は 'predict_price' 関数を使用して行えます。


## 最終提出データの作成

In [69]:
import pandas as pd
import numpy as np
import os
import pickle
import tensorflow as tf

# test_dfに対する推論
# test_IDが'spain_total'のデータのみを選択
test_df_spain = test_df[test_df['item_ID'] == 'spain_total'].copy()

# 欠損値処理 (trainと同じ処理を適用)
test_df_spain = test_df_spain.ffill()
test_df_spain = test_df_spain.bfill()

# 'item_ID' 列を削除
test_df_spain = test_df_spain.drop('item_ID', axis=1)

def create_additional_features_combined(df, is_test=False, train_df=None):
    """
    時間ベースの特徴量とラグ特徴量を作成。
    テストデータの場合は学習データと結合してラグ特徴量を正しく計算する。
    """
    df_features = df.copy()

    # インデックスがDatetimeIndexであることを確認し、必要なら変換
    if not isinstance(df_features.index, pd.DatetimeIndex) or df_features.index.tz is not None:
        df_features.index = pd.to_datetime(df_features.index, utc=True)

    # 時間ベースの特徴量
    df_features['hour'] = df_features.index.hour
    df_features['dayofweek'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['year'] = df_features.index.year
    df_features['quarter'] = df_features.index.quarter
    df_features['dayofyear'] = df_features.index.dayofyear
    try:
        df_features['weekofyear'] = df_features.index.isocalendar().week.astype(int)
    except AttributeError:
        df_features['weekofyear'] = df_features.index.weekofyear.astype(int)
    df_features['dayofmonth'] = df_features.index.day
    df_features['is_weekend'] = (df_features.index.dayofweek >= 5).astype(int)

    # ラグ特徴量の処理
    if is_test and train_df is not None:
        # テストデータの場合、学習データと結合してラグ特徴量を計算
        # 学習データの最後の部分とテストデータを結合
        combined_df = pd.concat([train_df, df_features], axis=0)

        # 結合データに対してラグ特徴量を計算
        combined_df['price_actual_lag_1'] = combined_df['price_actual'].shift(1)
        combined_df['price_actual_lag_24'] = combined_df['price_actual'].shift(24)
        combined_df['price_actual_lag_48'] = combined_df['price_actual'].shift(48)
        combined_df['price_actual_lag_168'] = combined_df['price_actual'].shift(168)
        combined_df['price_actual_rolling_mean_24'] = combined_df['price_actual'].rolling(window=24).mean()

        # テストデータ部分のみを抽出
        df_features = combined_df.loc[df_features.index]

        # price_actual列を削除（テストデータには存在しないため）
        if 'price_actual' in df_features.columns:
            df_features = df_features.drop('price_actual', axis=1)

    elif not is_test and 'price_actual' in df_features.columns:
        # 学習データの場合
        df_features['price_actual_lag_1'] = df_features['price_actual'].shift(1)
        df_features['price_actual_lag_24'] = df_features['price_actual'].shift(24)
        df_features['price_actual_lag_48'] = df_features['price_actual'].shift(48)
        df_features['price_actual_lag_168'] = df_features['price_actual'].shift(168)
        df_features['price_actual_rolling_mean_24'] = df_features['price_actual'].rolling(window=24).mean()

    # 欠損値処理
    df_features = df_features.ffill()
    df_features = df_features.bfill()

    # まだNaNが残っている場合は0で埋める
    df_features = df_features.fillna(0)

    return df_features

# 保存したパイプラインを読み込み
pipeline_load_path = data_path + 'price_prediction_pipeline.pkl'
loaded_pipeline = None
loaded_selected_features = None

if os.path.exists(pipeline_load_path):
    try:
        with open(pipeline_load_path, 'rb') as f:
            loaded_pipeline = pickle.load(f)

        loaded_pca = loaded_pipeline['pca']
        loaded_pca_scaler = loaded_pipeline['pca_scaler']
        loaded_y_scaler = loaded_pipeline['y_scaler']
        loaded_selected_features = loaded_pipeline['selected_features']
        loaded_seq_length = loaded_pipeline['seq_length']

        print(f"\nパイプラインをロードしました。学習時に選択された特徴量 ({len(loaded_selected_features)}個): {loaded_selected_features}")

    except Exception as e:
        print(f"パイプラインのロードに失敗しました: {e}")
        loaded_pipeline = None
else:
    print(f"エラー: パイプラインファイル '{pipeline_load_path}' が見つかりません。")

if loaded_pipeline is not None and loaded_selected_features is not None:
    # 学習データの特徴量エンジニアリング済みデータが必要
    # dfに対して同じ特徴量エンジニアリングを適用
    if 'df' in locals() or 'df' in globals():
        # 学習データに特徴量エンジニアリングを適用（price_actualラグを含む）
        df_featured_train = create_additional_features_combined(df.copy(), is_test=False)

        # テストデータに特徴量エンジニアリングを適用（学習データを使ってラグを計算）
        test_df_featured = create_additional_features_combined(
            test_df_spain.copy(),
            is_test=True,
            train_df=df.copy()
        )

        print(f"テストデータの特徴量エンジニアリング完了。形状: {test_df_featured.shape}")

        # NaN値の確認
        nan_count = test_df_featured.isnull().sum().sum()
        print(f"テストデータのNaN値の総数: {nan_count}")

        if nan_count > 0:
            print("NaN値が残っているカラム:")
            nan_columns = test_df_featured.isnull().sum()
            print(nan_columns[nan_columns > 0])

            # 残りのNaN値を0で埋める
            test_df_featured = test_df_featured.fillna(0)
            print("残りのNaN値を0で埋めました。")

        try:
            # 学習時に選択された特徴量のみを抽出
            test_data_aligned = test_df_featured[loaded_selected_features]
            print(f"テストデータ ({test_data_aligned.shape[1]}個の特徴量) のカラム順序を合わせました。")

            # 最終的なNaN確認
            final_nan_count = test_data_aligned.isnull().sum().sum()
            print(f"最終的なNaN値の総数: {final_nan_count}")

            if final_nan_count > 0:
                print("警告: まだNaN値が残っています。さらに処理します。")
                test_data_aligned = test_data_aligned.fillna(method='ffill').fillna(method='bfill').fillna(0)

        except KeyError as e:
            print(f"エラー: 学習時に選択された特徴量 '{e}' がテストデータに見つかりません。")
            test_data_aligned = None

        if test_data_aligned is not None:
            # モデルをロード
            lstm_model_path = data_path + 'lstm_model.h5'
            try:
                loaded_model = tf.keras.models.load_model(
                    lstm_model_path,
                    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
                )
                print("モデルを正常にロードしました。")

                # テストデータに前処理を適用
                test_data_scaled = loaded_pca_scaler.transform(test_data_aligned)
                test_data_pca = loaded_pca.transform(test_data_scaled)
                print(f"テストデータにスケーリングとPCAを適用しました。形状: {test_data_pca.shape}")

                # 学習データのPCA結果と結合
                if 'X_pca' in locals() or 'X_pca' in globals():
                    combined_data_pca = np.concatenate([X_pca, test_data_pca])
                    print(f"データを結合しました。形状: {combined_data_pca.shape}")

                    # 推論用シーケンスの作成
                    X_inference = []
                    start_index_test = len(X_pca)

                    for i in range(len(test_data_pca)):
                        seq_start = start_index_test + i - loaded_seq_length
                        seq_end = start_index_test + i

                        if seq_start >= 0:
                            input_sequence = combined_data_pca[seq_start:seq_end]
                            X_inference.append(input_sequence)

                    if X_inference:
                        X_inference = np.array(X_inference)
                        print(f"推論用シーケンスデータ形状: {X_inference.shape}")

                        # 予測実行
                        scaled_predictions = loaded_model.predict(X_inference)
                        predictions = loaded_y_scaler.inverse_transform(scaled_predictions).flatten()

                        # 提出ファイル作成
                        valid_test_indices = test_df_spain.index[len(test_df_spain) - len(predictions):]
                        submission_df = pd.DataFrame({
                            'id': [ts.strftime('%Y-%m-%d %H:%M:%S') + '_spain_total' for ts in valid_test_indices],
                            'price_actual': predictions
                        })

                        submission_path = data_path + 'submission.csv'
                        submission_df.to_csv(submission_path, header=False, index=False)

                        print(f"\n推論結果を '{submission_path}' に保存しました。")
                        print("提出ファイルの先頭:")
                        display(pd.read_csv(submission_path, header=None).head())

                    else:
                        print("エラー: 予測のための入力シーケンスが作成できませんでした。")
                else:
                    print("エラー: 学習データのPCA結果 (X_pca) が見つかりません。")

            except Exception as e:
                print(f"モデルのロードまたは推論に失敗しました: {e}")
    else:
        print("エラー: 学習データ (df) が見つかりません。")
else:
    print("パイプラインがロードされなかったため、推論を実行できません。")


パイプラインをロードしました。学習時に選択された特徴量 (30個): ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_other', 'total_load_actual', 'valencia_wind_speed', 'madrid_wind_speed', 'bilbao_pressure', 'bilbao_wind_speed', 'bilbao_wind_deg', 'bilbao_clouds_all', 'barcelona_wind_speed', 'seville_pressure', 'seville_wind_deg', 'hour', 'dayofweek', 'month', 'quarter', 'dayofyear', 'weekofyear', 'is_weekend', 'price_actual_lag_1', 'price_actual_lag_24', 'price_actual_lag_48', 'price_actual_lag_168', 'price_actual_rolling_mean_24']


<ipython-input-69-245eee2e3577>:72: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_features = df_features.ffill()


テストデータの特徴量エンジニアリング完了。形状: (8760, 105)
テストデータのNaN値の総数: 0
テストデータ (30個の特徴量) のカラム順序を合わせました。
最終的なNaN値の総数: 0
モデルを正常にロードしました。
テストデータにスケーリングとPCAを適用しました。形状: (8760, 15)
データを結合しました。形状: (35040, 15)
推論用シーケンスデータ形状: (8760, 24, 15)
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

推論結果を '/content/drive/MyDrive/ML/Signate_1634/submission.csv' に保存しました。
提出ファイルの先頭:


,0,1
0,2018-01-01 00:00:00_spain_total,59.314342
1,2018-01-01 01:00:00_spain_total,58.704517
2,2018-01-01 02:00:00_spain_total,58.757313
3,2018-01-01 03:00:00_spain_total,60.397415
4,2018-01-01 04:00:00_spain_total,61.126366


from matplotlib import pyplot as plt
_df_0[1].plot(kind='hist', bins=20, title=1)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby(0).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2[1].plot(kind='line', figsize=(8, 4), title=1)
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3[0].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x=1, y=0, inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)